In [1]:
from deap import base, creator, algorithms, tools, gp
from sklearn.metrics import mean_absolute_error
import operator
import random
import numpy as np

In [2]:
toolbox = base.Toolbox()

In [3]:
# Input datasets.
x = [(0, 10), (1, 9), (2, 8), (3, 7), (4, 6), (5, 5), (6, 4), (7, 3), (8, 2), (9, 1)]
y = [90, 82, 74, 66, 58, 50, 42, 34, 26, 18]

In [4]:
# Split the x into x0 and x1.
x0 = [val[0] for val in x]
x1 = [val[1] for val in x]

In [5]:
# Custom div function.
def div(a, b):
    try:
        return a/b
    except:
        return 0

In [6]:
# Function to evaluate a solution.
def eval_func(ind, input0, input1, expected_output):
    func_eval = toolbox.compile(expr=ind)
    predictions = list(map(func_eval, input0, input1))
    return abs(mean_absolute_error(expected_output, predictions)),

In [7]:
# Function to evaluate a solution.
def eval_func(ind, input0, input1, expected_output):
    func_eval = toolbox.compile(expr=ind)
    predictions = list(map(func_eval, input0, input1))
    return abs(mean_absolute_error(expected_output, predictions)),

In [8]:
# Select number of inputs and define operators.
pset = gp.PrimitiveSet('Main', 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(div, 2)

In [9]:
# Assign names to the arguments.
pset.renameArguments(ARG0='x0', ARG1='x1')
pset.addEphemeralConstant('R', lambda: random.randint(0, 10))

In [10]:
# Try to minimize usign a tree structure.
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register('select', tools.selTournament, tournsize=2)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('mutate', gp.mutNodeReplacement, pset=pset)
toolbox.register('evaluate', eval_func, input0=x0, input1=x1, expected_output=y)
toolbox.register('compile', gp.compile, pset=pset)

toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("min", np.min)
stats.register("avg", np.mean)
stats.register("std", np.std)

In [11]:
# Get best historic five.
hof = tools.HallOfFame(5)
# Assign a population of 10.
pop = toolbox.population(n=10)

# Run ea simple.
results, log = algorithms.eaSimple(pop, toolbox, cxpb=1.0, mutpb=0.1, ngen=10, stats=stats, halloffame=hof)

# Print the operation and MAE for each one of the best five in the hall of fame.
for index, ind in enumerate(hof):
    print(ind)
    print('Results of the best {} value: {}'.format(index+1, toolbox.evaluate(ind, input0=x0, input1=x1, expected_output=y)[0]))

gen	nevals	max    	min    	avg    	std    
0  	10    	6485.72	22.8571	703.519	1927.58
1  	10    	564.917	18.2857	102.794	155.026
2  	10    	1798   	38.3   	228.281	523.366
3  	10    	53.7401	32.46  	46.6309	7.76338
4  	10    	53.5746	33     	42.8603	6.26052
5  	10    	235.9  	37.5857	77.5561	65.7126
6  	10    	56.54  	14.3886	41.7549	10.7573
7  	10    	52.4371	12.1714	32.8123	13.6654
8  	10    	45.68  	8.9    	29.6267	14.4443
9  	10    	42.9507	6.8    	23.6508	12.3144
10 	10    	45     	8.24286	19.3514	12.1496
add(x0, add(add(x0, x0), sub(mul(x1, x1), 0)))
Results of the best 1 value: 6.8
add(x0, add(add(add(x0, x0), sub(x1, x1)), sub(mul(add(0, sub(x1, mul(1, div(x0, 7)))), x1), 0)))
Results of the best 2 value: 8.242857142857144
add(x0, add(x0, sub(mul(x1, x1), 0)))
Results of the best 3 value: 8.9
add(x0, add(add(x0, sub(x1, x1)), sub(mul(x1, x1), 0)))
Results of the best 4 value: 8.9
add(x0, add(add(sub(mul(add(0, x0), x1), 0), x0), sub(mul(x1, x1), 0)))
Results of the best 5 value